# Abstract

We demo a text classification approach based on an open dataset from https://world.openfoodfacts.org/. The goal of our talk is to categorize food items with various tags, where a particular food item is defined by a product's name, a generic name as well as a brand. For classification, we use facebooks NLP model fasttext, which provides a text classification model based on word embeddings as well as character n-gram embeddings. In a first experiment, we only use a single tag and remove additional ones from each data point. In this case, the evaluation is straight forward. However, since some classes are more closely related than others, we don't want to evaluate predictions in a binary manner as one would typically do. To this end, we implement a similarity concept and a multilabel classification approach. Additionally, we present some applications of a standardized food catalog, for instance search and recommendations.

In [31]:
!pwd

/Users/evelyn.trautmann/projects/pydata/evaluate_supervised


In [26]:
import pandas as pd
import numpy as np
from fasttext import *
%matplotlib inline
import evaluation_metrics

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import scipy.spatial.distance as sd
import sklearn.metrics.pairwise as pw
import re

# Load Data

The following classification is based on an open dataset from https://world.openfoodfacts.org/

In [30]:
df = pd.read_csv("data/foodcategories_single_label.csv.zip", sep = "\x01", compression="zip")
df.en_tags = df.en_tags.apply(eval)
df = df[df.en_tags.str.len()>0]

df.count()

product_name            324820
generic_name             83152
brands                  269465
categories              329303
categories_tags         329303
origins                  47351
manufacturing_places     73767
labels                  118779
emb_codes                52104
countries               329099
main_category           329297
en_tags                 329303
dtype: int64

In [28]:
label, freq = np.unique(np.concatenate(df.en_tags.values), return_counts = True)
sr_labels = pd.Series(index=label, data=freq).sort_values()


We want to classify tags based on product name, generic name and brand. In the first dataset we have a single tag assigned to each data point.

In [29]:
df.en_tags.str.len().value_counts()

1    329303
Name: en_tags, dtype: int64

# Extract Train and Test Data

In [5]:
#generate features
feature_cols = ['product_name', 'generic_name', 'brands']
assert(df[df.en_tags.str.len()==0].shape[0]==0)
X = df[feature_cols].fillna("").apply(lambda x: " ".join(x), axis = 1)
y = df.en_tags

In [6]:
# preprocess and split train and test date
X = X.str.lower().apply(lambda x: re.sub(r'[^\w\s]','',x))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

dftrain = y_train.apply(lambda x: " ".join(["__label__" + y
                    for y in x])) + " " + X_train

In [7]:
dftrain_emb = y_train.apply(lambda x: " ".join(x)) + " " + X_train
dftrain_emb.to_csv(
    "train_emb.csv", index = False, sep=";")

In [8]:
dftest = y_test.apply(lambda x: " ".join(["__label__" + y
                    for y in x])) + " " + X_test


In [9]:
dftrain.to_csv(
    "train.csv", index = False, sep=";")
dftest.to_csv(
    "test.csv", index = False, sep=";")

# Train

For classification we use fasttext, which can be either executed on command line or 

In [10]:
model_singlelabel = train_supervised("train.csv",
                autotuneValidationFile = "test.csv",
                autotuneDuration = 1000)

model_singlelabel.save_model("model_singlelabel.bin")
# !/Users/evelyn.trautmann/repos/fasttext3/fastText/fasttext supervised \
#     -input "train.csv" -output model_singlelabel \
#     -autotune-validation "test.csv" -autotune-duration 1000

In [13]:
!/Users/evelyn.trautmann/repos/fasttext3/fastText/fasttext dump model_singlelabel.bin args

dim 69
ws 5
epoch 18
minCount 1
neg 5
wordNgrams 3
loss softmax
model sup
bucket 10000000
minn 0
maxn 0
lrUpdateRate 100
t 0.0001


In [14]:
!/Users/evelyn.trautmann/repos/fasttext3/fastText/fasttext test model_singlelabel.bin test.csv 1

N	108669
P@1	0.783
R@1	0.783


In [16]:
model = load_model("model_singlelabel.bin")

In [15]:
model_emb = train_unsupervised("/Users/evelyn.trautmann/projects/openfood/train_emb.csv")

In [18]:
# model = train_supervised(input="/Users/evelyn.trautmann/projects/openfood/train.csv", 
#                          autotuneValidationFile="test.csv")

# Test

In [16]:
model_singlelabel.test("test.csv", k=1)

(108669, 0.7834617048100194, 0.7834617048100194)

In [17]:
df_test = X_test.to_frame()
df_test.columns = ["feature"]

# determine number of labels
df_test = df_test.join(y_test)

K = int(df_test.en_tags.str.len().quantile(0.75))
K

1

# Single Label Evaluation 

In [18]:
df_test["prediction"] = df_test.feature.apply(lambda x: model_singlelabel.predict(x))

df_test["label_predicted"] = df_test.prediction.str[0].str[0].str.replace("__label__","")
df_test["confidence"] = df_test.prediction.str[1].str[0]
df_test["truth"] = df_test.en_tags.str[0].str.replace("__label__","")


In [19]:
report = classification_report(df_test.truth, df_test.label_predicted)
print(report)

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                                 precision    recall  f1-score   support

            alcoholic-beverages       0.82      0.82      0.82      2683
                     baby-foods       0.81      0.71      0.76       497
                   bee-products       1.00      0.58      0.73        19
                          beers       0.00      0.00      0.00         5
                      beverages       0.53      0.51      0.52        51
             biscuits-and-cakes       0.67      0.52      0.58       875
                     breakfasts       0.89      0.88      0.88      5242
                          cakes       0.56      0.41      0.47       714
                        candies       0.77      0.28      0.41       133
                  canned-fishes       0.76      0.75      0.75       907
                   canned-foods       0.37      0.12      0.18       115
       canned-plant-based-foods       0.53      0.41      0.47       138
              carbonated-drinks       0.66      0.

In [20]:
# assertion no empty tags
assert(len(df_test.en_tags[df_test.en_tags.str.len()==0])==0)

# Similarity

Sometimes the model is predicting other than the ground truth but still makes reasonable class assignments. To distinguish the reasonable assignments from the actually wrong classifications we introduce similarities between classes to count them into the accurate predicftions.

Example
Consider followin common products
spaghetti bolognese, linguine bolognese, chicken biryani
Given that the first two are pretty similar and the latter very different, a possible similarity matrix S could look like



|Labels | spaghetti bolognese | linguine bolognese |chicken biryani|
|---------|:------------------|:-------------------|:--------------|
|spaghetti bolognese | 1 | 0.9 | 0 |
| linguine bolognese | 0.9 | 1 | 0 |
| chicken biryani | 0 | 0 | 1 |


If we want to include now similarity into calculation of precision and recall, we have the nominator containing not only

#[ Actual = spaghetti bolognese, Predicted = spaghetti bolognese ]

but

#[ Actual = spaghetti bolognese, Predicted = spaghetti bolognese ] * 1 + 
#[ Actual = spaghetti bolognese, Predicted = linguine bolognese ] * 0.9.

Hence diagonal entries of confusion matrix 

$$ C_{ii} = \#[Actual=i,Predicted=i]$$

become 

$$ C_{ii} = \sum_j \#[Actual=i,Predicted=j]* S_{ij} $$

That means, we count in all similar predictions weighted by degree of similatity.
Technically that means we take the diagonal entris of

$$\hat{C}=C∗S\hat{C} = C * S \hat{C}=C∗S$$


Where * denotes matrix product. Denominator remains the same: Row sum in case of precision, column sum in case of recall. Here we don't weight by similar entries since for denominator only the exact number of Actuals (resp. Predicted ) counts.

In [21]:

truth = ["truth"]
predictions = ["label_predicted"]
classes = list(set(np.concatenate(df_test[predictions + truth].apply(tuple).apply(list).values)))

%time confusion = evaluation_metrics.get_confusion(df_test, classes, K, truth, predictions)


compute diffecence sets
compute intersectiosn
compute onehot vectors
CPU times: user 7min 43s, sys: 11 s, total: 7min 54s
Wall time: 7min 54s


In [22]:
class_vecs = pd.Series({cl:model_emb.get_sentence_vector(cl) for cl in classes})

Dst = class_vecs.apply(lambda x: class_vecs.apply(lambda y: sd.euclidean(x,y)))

mask = Dst > 0.5
Dst[mask]=np.nan
Dst.apply(np.argmax)[Dst.apply(np.argmax)>0.0]

/usr/local/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:56: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


TypeError: '>' not supported between instances of 'numpy.str_' and 'float'

In [28]:
Dst[mask].count().value_counts()

0    80
dtype: int64

In [29]:
S = np.exp(-Dst**2).fillna(0)

In [31]:

df_report = evaluation_metrics.get_report(confusion, S)
df_report.dropna().sort_values(by="f1score", ascending = False)[:10]

,precision,recall,f1score,support
fish-and-meat-and-eggs,0.949606,0.955626,0.952607,631.0
farming-products,0.941061,0.958000,0.949455,500.0
sauces,0.934431,0.928431,0.931421,5024.0
olives,0.943761,0.914821,0.929066,587.0
chickens,0.938662,0.911389,0.924825,1282.0
fermented-milk-products,0.924838,0.916728,0.920765,4731.0
hams,0.932464,0.906040,0.919062,1341.0
poultries,0.895613,0.913071,0.904258,624.0
olive-tree-products,0.920337,0.858392,0.888286,1144.0
breakfasts,0.874285,0.895817,0.884920,5116.0


In [32]:
evaluation_metrics.get_summary(df_report, confusion, S)

,precision,recall,f1-score,support
micro avg,0.803342,0.803342,0.803342,108670.0
macro avg,0.639211,0.730071,0.681627,108670.0
weighted avg,0.820437,0.803342,0.811799,108670.0


# Multilabel Evaluation 

In [33]:
#load data
df = pd.read_csv("data/foodcategories_3labels.csv.zip", sep = "\x01", compression="zip")
df.en_tags = df.en_tags.apply(eval)
df = df[df.en_tags.str.len()>0]

In [34]:
y = df.en_tags
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

dftrain = y_train.apply(lambda x: " ".join(["__label__" + y
                    for y in x])) + " " + X_train
dftest = y_test.apply(lambda x: " ".join(["__label__" + y
                    for y in x])) + " " + X_test


In [35]:
dftrain.to_csv(
    "train_3label.csv", index = False, sep=";")
dftest.to_csv(
    "test3label.csv", index = False, sep=";")

In [38]:
train_supervised??

In [ ]:
model_3label = train_supervised("train_3label.csv",
                autotuneValidationFile = "test.csv",
                autotuneDuration = 1000)

# !/Users/evelyn.trautmann/repos/fasttext3/fastText/fasttext supervised \
#     -input "train_3label.csv" -output model_3label \
#     -autotune-validation "test3label.csv" -autotune-duration 1000

In [41]:
model_3label.save_model("model_3label.bin")

In [42]:
model = load_model("model_3label.bin")

Let \{$T_1^{(1)}, ...,T_M^{(1)}$\},...,\{$T_1^{(N)}, ...,T_M^{(N)}$\} be our ground truth, 
\{$P_1^{(1)}, ...,P_M^{(1)}$\},...,\{$P_1^{(N)}, ...,P_M^{(N)}$\} the prediction.

on diagonal we count the events of truth matching prediction

$\sum_{i=1}^N |\{T_1^{(i)}, ...,T_M^{(i)}\}\cap \{P_1^{(i)}, ...,P_M^{(i)}\}|$

so for each diagonal entry we count over all data points (N) and all predictions (M) the total amount when truth is matching prediction.

$C_{kk} =\sum_{i=1}^N \sum_{m=1}^M \chi{1}\{T_m^{(i)}=k,P_m^{(i)}=k\} $

in contrast on off-diagonal we count all events where truth was not covered by prediction.
For the off-diagonal we take only the difference sets into account. If truth for data point i was for example {fish, curry, rice-dish} and prediction was {fish, curry, soup }, we only count 
$T^i$ = {rice_dish} and $P^i$ = {soup} for the off-daigonal. Formally we do the following transformation:

$\{\tilde{T}_m^{(i)}\}_{q=1,..M1} = \{T_m^{(i)}\}_{m=1,..M}\setminus \{P_m^{(i)}\}_{m=1,..M}$ 

$\{\tilde{P}_m^{(i)}\}_{p=1,..M2} = \{P_m^{(i)}\}_{m=1,..M}\setminus \{T_m^{(i)}\}_{m=1,..M}$

Eventually we count all events of the ground truth that were not captured in predictions, which results in

$C_{kl} =\sum_{i=1}^N \sum_{m1=1}^{M1} \sum_{m2=1}^{M2}
\frac{1}{M1}\chi{1}\{\tilde{T}_{m1}^{(i)}=k,\tilde{P}_{m2}^{(i)}=l\} $

Once we have computed a multi-class - multi-label confusion matrix the classification report is straight forward.

Precision for class k is computed as

$Pr_k=\frac{C_{kk}}{\sum_j C_{kj}}$

and Recall

$R_k=\frac{C_{kk}}{\sum_j C_{jk}}$



In [43]:
df_test = X_test.to_frame()
df_test.columns = ["feature"]

# determine number of labels
df_test = df_test.join(y_test)

K = int(df_test.en_tags.str.len().quantile(0.75))
K

3

In [44]:
df_test["prediction"] = df_test.feature.apply(lambda x: model_3label.predict(x, k=K))

for k in range(K):
    df_test["label_predicted%i" %k] = df_test.prediction.str[0].str[k].str.replace("__label__","")
    df_test["confidence%i" %k] = df_test.prediction.str[1].str[k]
    df_test["truth%i" %k] = df_test.en_tags.str[k].str.replace("__label__","")


In [46]:
truth = list()
predictions = list()
for k in range(int(K)):
    truth.append("truth%i" %k)
    predictions.append("label_predicted%i" %k)
classes = list(set(np.concatenate(df_test[predictions + truth].apply(tuple).apply(list).values)))


In [48]:
%time confusion = evaluation_metrics.get_confusion(df_test, classes, K, truth, predictions)

CPU times: user 7min 53s, sys: 10.9 s, total: 8min 4s
Wall time: 8min 5s


In [49]:
df_report = evaluation_metrics.get_report(confusion)
df_report.dropna().sort_values(by="f1score", ascending = False)[:10]

,precision,recall,f1score,support
cooked-pressed-cheeses,0.945312,0.981346,0.962992,616.500000
prepared-meats,0.952300,0.943407,0.947832,5925.333333
mustards,0.913043,0.977433,0.944142,709.000000
hams,0.940639,0.941954,0.941296,1312.166667
farming-products,0.936293,0.945726,0.940986,512.833333
honeys,0.960664,0.921851,0.940858,1192.166667
fermented-milk-products,0.940730,0.937072,0.938898,9129.500000
fish-and-meat-and-eggs,0.957547,0.917629,0.937163,663.666667
meats,0.963753,0.908365,0.935240,9396.000000
poultries,0.948444,0.919695,0.933849,2320.333333


In [50]:
evaluation_metrics.get_summary(df_report, confusion)

,precision,recall,f1-score,support
micro avg,0.829391,0.829391,0.829391,293144.0
macro avg,0.704026,0.788582,0.743909,293144.0
weighted avg,0.840394,0.829391,0.834856,293144.0


# Combining Multilabel Classification with Near Accuracy 

In [54]:
class_vecs = pd.Series({cl:model_emb.get_sentence_vector(cl) for cl in classes})

Dst = class_vecs.apply(lambda x: class_vecs.apply(lambda y: sd.euclidean(x,y)))

mask = Dst > 0.5
Dst[mask]=np.nan
# show entries with highes distance <= 0.5 for each class
Dst.apply(np.argmax)

tomato-sauces                                         tomato-sauces
meals-with-meat                                     meals-with-meat
milks                                                         milks
sweeteners                                               sweeteners
pastries                                                   pastries
baby-foods                                               baby-foods
milk-substitute                                     milk-substitute
legume-seeds                                           legume-seeds
potato-crisps                                         potato-crisps
dairies                                                     dairies
fats                                                           fats
fishes                                                      seafood
fruits-based-foods                               fruits-based-foods
dairy-desserts                                       dairy-desserts
confectioneries                                 

In [55]:
S = np.exp(-Dst**2).fillna(0)
df_report = evaluation_metrics.get_report(confusion, S)
df_report.dropna().sort_values(by="f1score", ascending = False)[:10]

,precision,recall,f1score,support
cooked-pressed-cheeses,0.945312,0.981346,0.962992,616.500000
prepared-meats,0.952300,0.943407,0.947832,5925.333333
mustards,0.913043,0.977433,0.944142,709.000000
hams,0.940639,0.941954,0.941296,1312.166667
farming-products,0.936293,0.945726,0.940986,512.833333
honeys,0.960664,0.921851,0.940858,1192.166667
fermented-milk-products,0.940979,0.937203,0.939087,9129.500000
fish-and-meat-and-eggs,0.957547,0.917629,0.937163,663.666667
meats,0.963753,0.908365,0.935240,9396.000000
poultries,0.948444,0.921113,0.934579,2320.333333


In [56]:
evaluation_metrics.get_summary(df_report, confusion, S)

,precision,recall,f1-score,support
micro avg,0.83289,0.83289,0.83289,293144.0
macro avg,0.71257,0.795681,0.751836,293144.0
weighted avg,0.842982,0.83289,0.837906,293144.0
